## SPUR Densification Survey- Natural Langugage Processing Project
notebook author: Emily Robitschek

#### Background information on survey: 
This survey was conducted by the SPUR group at ETH: https://spur.ethz.ch/

#### Hypothesis/Research question: 
Can we use the open text field responses from densification surveys to infer something about the motivations behind why individuals responded the way they did to the densification survey?

#### Detailed notes and pre-processing based on preliminary inspection of the corpus
Initially, the raw data from the Chicago survey text responses (see Figure 1 for examples) was examined to identify potential caveats and preprocessing steps needed to properly prepare the data for analysis. In the process it was observed that:
1)	Response length varies greatly, with shorter responses more often (but not always) being "throw away" responses that are either somewhat random (e.g. "Fyfgfx" or "Bzsbsnsbsn") or unrelated to the survey or displaying a sort of bored, uncertain or apathetic attitude (or maybe they were just tired of filling out the survey at that point - the interpretation is difficult). Examples include: "Idk", "Nope", "Favor", "Nigh", "40", "I’m not yo I’m not talking to you".

2)	Even in substantive responses, spelling errors can significantly change the meanings of the responses. For instance, in the case of the response "Because it would be more far to people that don't have much") - this person clearly meant "fair" not "far", and in the case of "I think the 10 percent is to high of a yearly increase" - this person clearly meant "too" not "to". Also, slang words appear quite often within responses so filtering/interpreting those terms will add a potentially challenging element as well. 

For the variations described in point 1 of the discussion of the raw data, these patterns might be informative in and of themselves if they relate reliably and systematically to preferences for certain densification policies determine from the more quantitative aspects of the survey. These responses could even potentially speak to sentiments like lack of trust or lack of knowledge which are likely to influence policy views. However, the interpretation of such an analysis, especially if no specific words relevant to the policies are mentioned, would be challenging as the response character is confounded by other factors like sampling biases and the difficulty/impossibility of inferring the true motivations behind the response. For example, if someone says "I don't know" it is not clear if they don't know which policy to pick or if they don't know what else to say for the text response or if they are simply tired of filling out the survey and want it to be over. For point 2, greater length is not always directly indicative of a more meaningful response but in our case we might address some of the issues of varying response lengths by setting an initial cutoff.


## Stage 1: Pre-processing and analysis of response length

To ensure that the model has the best possible input data and therefore the best chance of generating useful and interpretable outputs in the form of topics and keywords related to densification, quality control and preprocessing was undertaken. Once this was completed, 4886 survey responses remained.

In [ ]:
## import necessary packages: 
import pandas as pd
import numpy as np

#graphing/visualization packages: 
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [ ]:
## Helper functions
def filter_response_df(response_df, open_text_column, min_char=0):
    """
    Takes in a dataframe for survey responses (response_df) and the column with the open text field of 
    interest (open_text_column) and a minimum character limit (automatically set at zero for no filtering) 
    and returns a dataframe with the null responses for that column filtered out and if min_char > 0, also 
    filters out responses shorter than or equal to the min_char limit.
    """
    #filter na responses
    response_df = response_df.dropna(subset=[open_text_column])
    #calculate character lengths of all non-NA responses
    response_char_length = []
    for i in range(0, len(response_df)):
        char_length = len(response_df[open_text_column][i])
        #if char_length < 10: ## if want to look at some of the responses for short character lengths
        #    print(response_df[open_text_column][i])
            #print(char_length)
        response_char_length.append(char_length)
    response_df['response_char_length'] = response_char_length
    if min_char > 0: 
        return response_df[response_df['response_char_length'] > min_char].copy()
    else: 
        return response_df

def encode_cat(df, column, ordinal=False): 
    if ordinal == True: 
        ord_enc = OrdinalEncoder()
        df[('%s_cat' % column)] = ord_enc.fit_transform(df[[column]])
    else: 
        df[column] = df[column].astype('category')
        df[('%s_cat' % column)] = df[column].cat.codes
    return df

## Read in data

### Generate dataframe to spell correct for cities besides Chicago all at once

In [ ]:
#load in survey dataframe
filepath = '../../../Data_Emily/Densification_Survey/df_spell_corr.csv'
df_to_spell_correct = pd.read_csv(filepath)
print(len(df_to_spell_correct))

#select only responses from the English-speaking cities 
df_to_spell_correct = df_to_spell_correct[df_to_spell_correct['city'].isin(['Chicago', 'New York', 'Los Angeles', 'London'])]
print(len(df_to_spell_correct))

#view column with text response
df_to_spell_correct['Q12.6']

#read in Chicago responses (already spell corrected prior to the other three cities)
chicago_spell_corrected_file = '../../../Data_Emily/Densification_Survey/Chicago_original_spellcor.csv'
chicago_sp_corr_df = pd.read_csv(chicago_spell_corrected_file, 
                                 encoding='latin-1', 
                                 skiprows=[1,2])

#merge data and see how many responses there are per city and what the data looks like
df_to_spell_correct = df_to_spell_correct.merge(chicago_sp_corr_df[['ResponseId', 'Q12.6_corrected']], how='left', left_on='ResponseId', right_on='ResponseId')
print(df_to_spell_correct.city.value_counts())
df_to_spell_correct[['ResponseId', 'Q12.6', 'Q12.6_corrected']].head()

#specify an output file to write out respondent id and responses in order to spell correct the responses 
#for the other three cities

#out_file = '../../../Data_Emily/Densification_Survey/df_short_responses_spell_correct.csv'
#df_to_spell_correct[['ResponseId', 'Q12.6', 'Q12.6_corrected', 'city']].to_csv(out_file)

### Read in survey and spell corrected responses to merge 

In [ ]:
#load in survey dataframe
filepath = '../../../Data_Emily/Densification_Survey/df_spell_corr.csv'
df_to_spell_correct = pd.read_csv(filepath)
print('Number of responses for all 6 cities in the survey: ', len(df_to_spell_correct))

#select only responses from the English-speaking cities 
df_to_spell_correct = df_to_spell_correct[df_to_spell_correct['city'].isin(['Chicago', 'New York', 'Los Angeles', 'London'])]
print('Number of responses for the 4 English-speaking cities in the survey: ', len(df_to_spell_correct))

## load in spell corrected responses: 
spellcorrected_file = '../../../Data_Emily/Densification_Survey/df_short_responses_spell_correct_v1.csv'
spell_corrected_df = pd.read_csv(spellcorrected_file, encoding='latin-1')

#fill the na values with a single letter so they can be included in the response length comparison across respondent characteristics
spell_corrected_df = spell_corrected_df.fillna(value='N')  

#merge the two dataframes
survey_response_df = df_to_spell_correct.merge(spell_corrected_df[['ResponseId', 'Q12.6_corrected']], 
                                                     how='left', left_on='ResponseId', right_on='ResponseId')

#specify column with corrected text
subset_col = 'Q12.6_corrected'

#run this function without filtering to generete the response_char_length parameter 
response_df = filter_response_df(survey_response_df, subset_col, min_char=0)
response_df = response_df.drop(columns=(['Unnamed: 0'])) # drop extra columns from filtering

print('Number of responses after merge and filter: ', len(response_df))
response_df.head(10)

#save to file: 
#response_df.to_csv('../../datasets/spur_survey_responses_english_spellcor.txt', index=False, sep='\t')

### Compute some summary statistics for response length across cities

In [ ]:
plt.figure(figsize=(4,4))

# Groupby by city
city_initial = response_df.groupby("city")

city_initial.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("City")
plt.ylabel("Number of responses")
plt.show()

# Summary statistic of all countries
city_initial['response_char_length'].describe().head()

### Correlations of response length and other survey variables
Because response length itself could vary based on respondent characteristics, the spearman correlation was computed between the response lengths (NaN responses were counted to have a length of 1 character, to try to account for bias in who even attempted to fill out the text response) and a subset of other metadata columns in the survey, including potential covariates of response length (e.g. age, income, minority status, location) before length filtering and after length filtering. Project location was encoded with a categorical encoder for inclusion. The resulting correlations were sorted in a descending manner for display.  

Note: 'Q6.1' is densif_proj_acceptance_preframe, 'Q7.5' is densif_proj_acceptance_postframe, 'Q7.6' is the percieved impact of the project on rent, 'Q7.7' is the percieved impact of the project on social cohesion  

In [ ]:
#encode some socio-demographic factors as a dummy variable for the correlation analysis:

#columns to include from survey
cols_to_include = [subset_col, 'frame', 'projectLocation',
                   'minority', 'linguistic_minority', 'religious_minority', 'social_class_minority', 
                   'sexual_orientation_minority', 'ethnic_minority', 'national_minority', 'other_minority', 
                   'disabled_minority',  'Q3.1', 'Q3.3', 'Q3.5', 'Q3.7', 'Q5.5', 'Q14.1_1', 'Q3.11', 'Q3.16', 
                   'Q4.1', 'Q5.1', 'Q6.5_3', 'Q6.5_6', 'Q6.5_9', 'Q6.5_10', 'Q6.11_1', 'Q6.15_5', 'Q6.15_6', 
                   'Q14.3', 'Q14.9', 'age', 'city', 'country', 
                   'response_char_length', 'Q6.1', 'Q7.5', 'Q7.6', 'Q7.7']


#create dictionary for mapping the column names to a readable/interpretable form based on the survey question
substantive_colnames_map = {'Q3.1': 'gender', 'Q3.3': 'age_numerical', 'Q3.5': 'income', 'Q3.7': 'citizenship', 'Q3.11': 'exp_housing_discrim',
                       'Q3.16': 'city_district', 'Q4.1': 'city_neighborhood', 'Q5.1': 'prof_situation', 'Q5.5': 'education', 
                       'Q6.5_3': 'fear_forced_out', 'Q6.5_6': 'seen_landlord_pressure', 'Q6.5_9': 'changes_in_community', 
                       'Q6.5_10': 'seen_renovations', 'Q6.11_1': 'neighborhood_connection', 'Q6.15_5': 'respect_rules', 
                       'Q6.15_6': 'neighbhorhood_crime', 'Q14.1_1': 'political_left_right', 
                       'Q14.3': 'env_pollution_impact', 'Q14.9': 'climate_concern'}

#create scales to map the likert style variables to a numerical scale (columns specified below)
mapping_likert = {'do not agree at all': -2, 'do not agree': -1, 'neutral': 0, 'agree': 1, 'fully agree': 2}
mapping_likert_neighborhood_connection = {'not connected at all': -2, 'not very connected': -1, 'neutral': 0, 'somewhat connected': 1, 'strongly connected': 2}
mapping_likert_climate_concern = {'Not concerned': -2, 'Rather not concerned': -1, 'Neither': 0, 'Quite concerned': 1, 'Very concerned': 2}

#create scale to map the age (categorical)
mapping_age = {'55+': 5, '45-54': 4, '35-44': 3, '25-34': 2, '18-24': 1}

#specify columns of each variable type
likert_cols = ['fear_forced_out', 'seen_landlord_pressure', 'changes_in_community', 'seen_renovations', 'respect_rules', 
               'neighbhorhood_crime']

dummy_cols = ['gender', 'citizenship', 'education', 'exp_housing_discrim', #'city_district', 'city_neighborhood', 
              'prof_situation', 'city', 'country']

response_df = response_df[['ResponseId'] + cols_to_include]
response_df = response_df.rename(columns=substantive_colnames_map)

#remap some columns
for col in likert_cols: 
    response_df[col] = response_df[col].map(mapping_likert)
response_df['climate_concern'] = response_df['climate_concern'].map(mapping_likert_climate_concern)
response_df['neighborhood_connection'] = response_df['neighborhood_connection'].map(mapping_likert_neighborhood_connection)
response_df['age'] = response_df['age'].map(mapping_age)

#also make a version of the dataframe with dummy variables: 
response_df_w_dummies = pd.get_dummies(response_df, columns=dummy_cols)
print(list(response_df.columns))
print(list(response_df_w_dummies.columns))
response_df_w_dummies.head()

In [ ]:
response_df = encode_cat(response_df, 'projectLocation', ordinal=False) #encode project location for correlation

cols = ['ResponseId', 'Q12.6_corrected', 'frame', 'projectLocation', 'minority', 'linguistic_minority', 'religious_minority', 
        'social_class_minority', 'sexual_orientation_minority', 'ethnic_minority', 'national_minority', 'other_minority', 'disabled_minority', 
        'age_numerical', 'income', 'political_left_right', 'city_district', 'city_neighborhood', 'fear_forced_out', 'seen_landlord_pressure', 
        'changes_in_community', 'seen_renovations', 'neighborhood_connection', 'respect_rules', 'neighbhorhood_crime', 'env_pollution_impact', 
        'climate_concern', 'age', 'response_char_length', 'gender_Female', 'gender_Male', 'gender_Non-binary', 'gender_Prefer not to say', 
        'citizenship_Dual citizenship', 'citizenship_EU citizen', 'citizenship_National', 'citizenship_Other', 'citizenship_US Born', 'citizenship_US Naturalization', 
        'education_Apprenticeship', "education_Bachelor's degree", 'education_Doctoral degree', 'education_Higher education entrance qualification', 
        'education_Higher technical examination', "education_Master's degree", 'education_No school certificate', 'education_Other', 'education_Secondary school certificate', 
        'exp_housing_discrim_Do not know', 'exp_housing_discrim_No', 'exp_housing_discrim_Yes', 'prof_situation_Employed (full-time)', 'prof_situation_Employed (part-time)', 
        'prof_situation_Homemaker', 'prof_situation_Not employed (unemployed)', 'prof_situation_Not working (disability/illness)', 'prof_situation_Not working (other reason)', 
        'prof_situation_Not working (retired)', 'prof_situation_Student', 'city_Chicago', 'city_London', 'city_Los Angeles', 'city_New York', 'country_United Kingdom', 'country_United States']
df_corr= response_df_w_dummies[cols]

#drop variables that are NA for all values
df_corr = df_corr.dropna(axis=1, how='all')
df_corr.head()

In [ ]:
#make heatmap of plot above
plt.figure(figsize=(1, 12))
corr = df_corr.corr(method='spearman')
ax = sns.heatmap(corr[["response_char_length"]].sort_values(by=["response_char_length"], ascending=False),
                 yticklabels=True,
                 robust=True, 
                 annot=True, 
                 cmap="YlGnBu")
title = ("Correlation heatmap of response length and potential covariates" )
plt.title(title)
file_name=('../../figures/' + title + ('_v2_feb2022_spearman.png'))
plt.savefig((file_name), dpi=300, bbox_inches='tight')

### Data exploration: Lengths of responses (non-NA or 1 character length responses) for response length cut-off
We can notice that response length varies greatly, with shorter responses more often (but not always) being "throw away" responses that are either somewhat random (e.g. "Fyfgfx" or "Bzsbsnsbsn") or unrelated to the survey or displaying a sort of bored, uncertain or apathetic attitude (or maybe they were just tired of filling out the survey at that point - the interpretation is difficult). Examples include: "Idk", "Nope", "Favor", "Nigh", "40", "I’m not yo I’m not talking to you". Greater response length is not always directly indicative of a more meaningful response but in our case we might address some of the issues of varying response lengths by setting an initial cutoff. 

In [ ]:
response_df = filter_response_df(response_df, subset_col, min_char=1) #filter nan and 1-character responses
response_df = response_df.reset_index()
response_df = response_df.drop(columns=(['index']))  # drop extra column from resetting the index
response_df["ln_response_char_length"] = response_df["response_char_length"].apply(lambda x: np.log(x))

print("There are %d entries for the text field in this dataset" % (len(response_df)))
response_df['response_char_length'].head()

In [ ]:
## Plot distribution of length of responses by city
#non-log data
plt.rcParams["figure.figsize"] = (12,6)
sns.histplot(data=response_df, x='response_char_length', bins=70, hue="city", alpha=0.3, element="step")
plt.xlabel("Character length of short-response for survey")
plt.ylabel("Frequency")
title = ("Histogram of character length of short-response for survey across sites")
plt.title(title)
file_name=('../../figures/' + title + ('_v2.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

#log data
plt.rcParams["figure.figsize"] = (12,4)
sns.histplot(data=response_df, x="ln_response_char_length", bins=70, hue="city", alpha=0.3, element="step")
plt.xlabel("ln(character length of short-response for survey)")
plt.ylabel("Frequency")
title = ("Histogram of ln(character length of short-response) across sites")
plt.title(title)
file_name=('../../figures/' + title + ('_v2.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

#non-log data for shorter responses
sns.histplot(data=response_df[response_df['response_char_length'] <40], x='response_char_length', bins=8, hue="city", alpha=0.3, element="step")
plt.xlabel("Character length of short-response for survey")
plt.ylabel("Frequency")
title = ("Distribution of character length of short-response for responses <40 characters across sites")
plt.title(title)
file_name=('../../figures/' + title + ('_v2.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

#non-log data catplot
g = sns.catplot(x='response_char_length', y="city",
                hue="city", 
                data=response_df,
                orient="h", height=6, aspect=2, palette="Set3",
                kind="violin", dodge=True, cut=0, bw=.2)

plt.xlabel("Character length of short-response for survey")
plt.ylabel("Frequency")
title = ("Distribution of character length of short-response for survey across sites")
plt.title(title)
file_name=('../../figures/' + title + ('_v2_violins.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')

#log data catplot
g = sns.catplot(x="ln_response_char_length", y="city",
                hue="city", 
                data=response_df,
                orient="h", height=6, aspect=2, palette="Set3",
                kind="violin", dodge=True, cut=0, bw=.2)

plt.xlabel("ln(character length of short-response)")
plt.ylabel("Frequency")
title = ("Distribution of ln(character length of short-response) for survey across sites")
plt.title(title)
file_name=('../../figures/' + title + ('_v2_violins.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')


In [ ]:
# cut out short responses for now of ~15 characters
min_char = 15 
response_df = filter_response_df(response_df_w_dummies, subset_col, min_char= min_char)  
response_df = response_df.reset_index()
response_df = response_df.drop(columns=(['index'])) # drop extra column from resetting the index

print("There are %d entries for the text field with less than %d characters in this dataset" % (len(response_df), min_char))
response_df['response_char_length'].head()

### make correlation heatmap for data after 15 character minimum to see if response length remains not strongly correlated with other survey variables

In [ ]:
plt.figure(figsize=(1, 12))
df_corr = response_df[cols]
df_corr = df_corr.dropna(axis=1, how='all')

corr = df_corr.corr(method='spearman')
ax = sns.heatmap(corr[["response_char_length"]].sort_values(by=["response_char_length"], ascending=False),
                 yticklabels=True,
                 robust=True, 
                 annot=True, 
                 cmap="YlGnBu")
title = ("Correlation heatmap of response length and potential covariates min_char=15" ) 
plt.title(title)
file_name=('../../figures/' + title + ('_v2_feb2022_spearman.png'))
plt.savefig((file_name), dpi=300, bbox_inches='tight')

### Export dataframe post filtering

In [ ]:
response_df = pd.read_csv('../../datasets/spur_survey_responses_english_spellcor.txt', sep='\t')
response_df = filter_response_df(response_df, subset_col, min_char= min_char)  
response_df = response_df.reset_index()
response_df = response_df.drop(columns=(['index'])) # drop extra column from resetting the index
#response_df.to_csv('../../datasets/spur_survey_response_filtered_df1.txt', index=False, sep='\t')

### Discussion of how the corpus will help answer the research question
In addition to the open text field, the survey had many other structured questions to gauge participants responses to and feelings about densification projects and intersecting concerns at the community level (e.g. cost of living) and at a more global level (e.g. climate change concerns). Some examples include: 

    1) The frame on whether the project was situated in respondent’s own neighborhood/district or somewhere else
    2) Q7.5 on general acceptance of such densification projects 
    (alternatively also Q6.1 which is the same question but was asked before the frame)
    3) Q7.6 on the perceived effect of such projects on rent
    4) Q7.7 on the perceived effect of such projects on social cohesion & composition within the neighborhood
    5) Q14.9 on climate change concern

In order to compare and interpret the results of the analysis of the text responses with other key survey elements, a subset of these questions will be used to stratify respondents into subpopulations (e.g. those who always accept densification projects versus those that always reject them).

In [ ]:
question = 'frame'
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (project location)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig(file_name, dpi=300, bbox_inches='tight')
plt.show()

question = 'Q6.1'
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (acceptance densif. projects before frame)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

question = 'Q7.5'
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (acceptance densif. projects after frame)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

question = 'Q7.6'
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (perceived effect on rent)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

question = 'Q7.7'
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (perceived effect on social cohesion)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

question = 'Q14.9'
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (climate change concern)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

question = 'Q14.1_1' #survey question on political views
g = sns.displot(response_df, x=question, hue=question, palette="Set3", shrink=.8)
plt.xlabel(question)
plt.xticks(rotation=45)
plt.ylabel("Frequency")
title = ("Responses for %s (politcal leaning 0- most left and 7-most right)" % question)
plt.title(title)
file_name=('../../figures/survey_characteristics_bar/' + title + ('_v1_minchar15_bar.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Groupby by city
city = response_df.groupby("city")

plt.figure(figsize=(4,4))
city.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("City")
plt.ylabel("Number of responses above threshold")
plt.show()

# Summary statistic of all countries
city['response_char_length'].describe().head()

In [ ]:
#see if the dataframe wrote out properly
#test = pd.read_csv('../../datasets/spur_survey_response_filtered_df1.txt', sep='\t')
#test.head()